In [133]:
import numpy as np
import nltk
import pandas as pd
# nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import plotly
import plotly.express as px
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import matplotlib.pyplot as plt

# Playing with text and concatenation

In [66]:
words = ['quick', 'brown', 'fox', 'jumped']

In [67]:
concat_words = ''
for word in words:
    concat_words += word + ' '
concat_words

'quick brown fox jumped '

In [68]:
def concat_words(list):
    concat_words = ''
    for word in list:
        concat_words += word + ' '
    return concat_words

In [69]:
test = ['a','b','c','donkey', np.NaN]

In [70]:
test2 = [i for i in test if i is not np.nan]
test2

['a', 'b', 'c', 'donkey']

In [71]:
concat_words(test2)

'a b c donkey '

In [72]:
embed = [[1, 2, 3]]

In [73]:
embed

[[1, 2, 3]]

In [74]:
embed[0]

[1, 2, 3]

In [75]:
[1,2] + [3,4]

[1, 2, 3, 4]

In [76]:
[['a', 'b']] + [['c']]

[['a', 'b'], ['c']]

# Lemmatization

In [77]:
lemm = WordNetLemmatizer()

In [78]:
lemm.lemmatize('running', 'v')

'run'

In [79]:
lemm.lemmatize('reporting')

'reporting'

In [80]:
lemm.lemmatize('reports')

'report'

In [81]:
lista =[]
for idx, word in enumerate(words):
    lista.append(word)
lista

['quick', 'brown', 'fox', 'jumped']

In [82]:
processed1 = ['original','messages','sent', 'bank','subject'] 
processed2 = ['loved', 'banks', 'feet', 'reporting']

In [83]:
def make_lemma(list_of_words):
    list_to_return = []
    for idx, word in enumerate(list_of_words):
        if word == 'reporting':
            word = 'reports'
        if (word == 'pass' or word == 'blue'):
            continue
        list_to_return.append(lemm.lemmatize(word))
    return list_to_return

In [84]:
make_lemma(processed1)

['original', 'message', 'sent', 'bank', 'subject']

In [85]:
make_lemma(processed2)

['loved', 'bank', 'foot', 'report']

In [86]:
processed3 = ['loved', 'pass', 'blue', 'banks', 'feet', 'reporting', 'blue bayou']

In [87]:
words_to_skip = ['pass', 'blue']

In [88]:
def make_lemma(list_of_words):
    list_to_return = []
    for idx, word in enumerate(list_of_words):
        if word == 'reporting':
            word = 'reports'
        if word in words_to_skip:
            continue
        list_to_return.append(lemm.lemmatize(word))
    return list_to_return

In [89]:
processed3

['loved', 'pass', 'blue', 'banks', 'feet', 'reporting', 'blue bayou']

In [90]:
make_lemma(processed3)

['loved', 'bank', 'foot', 'report', 'blue bayou']

In [91]:
['string bing'] + ['bang lang']

['string bing', 'bang lang']

# Plotly

In [92]:
df = pd.DataFrame({
    'col1': [2, 2, 1, 3, 10, 5],
    'col2': [2, 1, 9, 8, 7, 4],
    'col3': [4, 1, 9, 4, 2, 3]
    })

In [93]:
df

,col1,col2,col3
0,2,2,4
1,2,1,1
2,1,9,9
3,3,8,4
4,10,7,2
5,5,4,3


In [94]:
df.iloc[1]

col1    2
col2    1
col3    1
Name: 1, dtype: int64

In [95]:
df.loc[[1]]

,col1,col2,col3
1,2,1,1


In [96]:
values = df.loc[[1]]
values

,col1,col2,col3
1,2,1,1


In [97]:
# fig = px.pie(df, values=df['col1'])
# fig.show()

In [98]:
# fig = px.pie(df, values=df.loc[0])
# fig.show()

In [99]:
# df.loc[2]

In [100]:
# fig = px.pie(df.T, values=2)
# fig.show()

# Looping over dataframe to process text

In [101]:
data = pd.DataFrame({
    'class': ['credit', 'loan', 'retail', 'credit'],
    'narrative': ['-- -- -- -- -- 1. ) XXXX XXXX, XXXX a purchase with EIO.com ( order # XXXX ) with 2 days shipping for the amount of {$57.00}. \n\n-- -- -- -- -- -2. ) Did not receive the product after a week, so I sent a followup email ; with the exact verbiage : " I paid for two days shipping and have not received my order \'\' on XXXX XXXX. \n\n-- -- -- -- -- 3. ) Company Responded on on XXXX XXXX : " Im sorry to inform you that due to unusually high order volume, your order has not shipped and XXXX not be for several weeks. XXXX has been out of stock since early XXXX due to high demand, although they are continuing to take orders and guaranteeing that we will receive orders we place with them in due time. As such we have masks on order but we do not have an exact shipping date from XXXX right now, however we can guarantee we will ship as soon as soon as XXXX delivers product to us. We have been getting small shipments and shipping those on a first come, first served basis, so we appreciate your patience as we fulfill these orders as quickly as we can. \n\nWe recommend keeping your order as you will lose your place in line if you cancel, and no other distributor has stock at the moment. If you do prefer to cancel, please note we ask that you do so via email, we will then cancel in accordance with our cancellation policy which was agreed to at checkout. \nXXXX, XXXX XXXX. \nElectronic Inventory Online -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- XXXX XXXX XXXX XXXX -- -- -- XXXX. ) XX/XX/XXXX, requested for the order to be canceled and a refund to be issued. \n\n-- -- -- 5. ) XX/XX/XXXX : XXXX canceled the order and sent a verification : XXXX XXXX XXXX Your order has been canceled Refunded Items XXXX XXXX Particulate Respirator, XXXX, XXXX eaches/pack 2 Refunded {$45.00} Subtotal {$45.00} Shipping {$12.00} Taxes {$0.00} Total {$57.00} USD Visa ( ending in XXXX XXXX {$57.00} Refund Shopify_payments - {$52.00} -- -- XX/XX/XXXX, Called in and disputed the amount. XXXX stated nothing needed to be submitted and they will address the issue. \n\n-- -- XX/XX/XXXX, XXXX recharged me for the same item after removing it on XX/XX/XXXX. Called back in to dispute this amount again. \n\nTRANSACTION REBILLMerchandiseRobert J. XXXX {$57.00} HTTPSEIOCOM.M, CA XXXX Purchased : Thu, XX/XX/2020 Posted : Wed, XXXX XXXX, 2020 Purchased By : XXXX XXXX Appears on the statement as : TRANSACTION REBILL HTTPSEIOCOM.M CA XXXX US XXXX - XX/XX/XXXX, followed up to see the status of the case. Submitted documentation showing XXXX canceled the order and was supposed to submit a refund. \n\n-- - XX/XX/XXXX, Called back in to speak to the manager on the case. He stated the dispute was ruled in favor of me, and the charge will be removed from my card. \n\n-- -- XX/XX/XXXX, Capital One removed the purchase off the bill. \nPURCHASE ADJUSTMENTMerchandiseRobert J. ... - {$57.00} Posted : Fri, XX/XX/2020 Purchased By : XXXX XXXX Appears on statement as : PURCHASE ADJUSTMENT XXXX XXXX -- XXXX XXXX Capital One recharges me the same amount. \nXX/XX/XXXX TRANSACTION REBILLMerchandiseRobert J. ... {$57.00} Purchased : Thu, XX/XX/2020 Posted : Mon, XX/XX/2020 Purchased By : XXXX XXXX Appears on statement as : TRANSACTION REBILL XXXX XXXX -- -- XX/XX/XXXX -- Called Capital One requested why this was recharge. They stated Visa ruled against my case. They pretended that they didn\'t remove the purchase or knew anything about the case manager ruling in my favor.',
                  '-- -- -- -- -- Forwarded message -- -- -- -- - From : XXXX XXXX XXXX Date : Tue, XX/XX/XXXXat XXXX XXXX Subject : Please Investigate Comenity Bank Retailers card scam To : XXXX Sent from my XXXX Hello my name is XXXX XXXX, I am being scammed by Comenity bank a credit card provider for companies The Childrens place, New York & Co. , Forever 21 and Victoria Secret. My original credit from XXXX was {$500.00} Comenity bank then lowers my limit to {$300.00} and began to charge overage fees along with late fees. I then began to pay close attention to my other cards to find that my limits were also changed on them as well incurring overages and late fees. \nI reached out to the company Comenity bank they stated that they would change my credit limit to its original limits but did not. I reached out to them again and told them I will not summit any payment until my accounts are corrected. Comenity bank credit cards has impacted my credit scores plummeted to a negative status. Im currently paying the price due to the corruption in which has affected me in detrimental way. \nI am now in debt over {$2000.00} due to the company charging overage fees as well as late fees even through COVID-19. The initial credit limits are fluctuating tremendously and the company charges major fees on each of those 4 accounts. They are not willing to correct my account and was nervous when I said I had an attorney, that is the reason Im reaching out to you. Im a XXXX  employee of 18yrs and this company is ruining my credit. PLZ HELP!!!!! \n\nMy name is XXXX XXXX, contact info is ( XXXX ) XXXX XXXX and/or XXXX. Thank you!!',
                 '-- -- - XXXX, XXXX, XX/XX/2020 Payment Histories Missing On My Credit Report -- -- Specialized Loan Servicing ( SLS ) has made the mistake to put my account under forbearance in XXXX 2020, without my authorization or knowledge ( As a matter of fact, I have automatic payment setup. In each month \'s XXXX, the monthly mortgage was paid in full ). \n\nI have noticed this issue ( my account was marked as " forbearance \'\' ) in XX/XX/2020 on my credit report ( when I tried to get a new home loan from another new bank ), I have contacted them immediately and asked them to fix this error and provide me a letter ( details please see below asks ). The " forbearance \'\' issue itself seemed to be fixed by now. However, on my credit report, the payment histories for XXXX, XXXX, XX/XX/2020 were all missing. My new bank will not be able to approve my new loan because of these issues of missing payment histories. \n\nI have contacted Specialized Loan Servicing more than 20 times since XX/XX/XXXX by phone to ask SLS to do 2 things : ( 1 ) report my XXXX, XXXX, XX/XX/2020 payment histories to XXXX, XXXX, XXXX  to fix the missing data issue. \n\n( 2 ) Provide me a letter with shows my account has never been under forbearance and my payment histories for all past 12 months.\n\nHowever, after waiting for more than 3 weeks, and countless emails / phone calls / talks with multiple supervisors, I have not been able to get either one of the above 2 things.\n\nWithout this issue fixed, My new bank can not process my new loan application, therefore I will need SLS \'s help immediately to get this fixed.',
                 'Make a point to try to make it to the homes']
    })

In [102]:
data

,class,narrative
0,credit,"-- -- -- -- -- 1. ) XXXX XXXX, XXXX a purchase..."
1,loan,-- -- -- -- -- Forwarded message -- -- -- -- -...
2,retail,"-- -- - XXXX, XXXX, XX/XX/2020 Payment Histori..."
3,credit,Make a point to try to make it to the homes


In [103]:
stopwords_list = stopwords.words('english') + list(string.punctuation)
stopwords_list += ["''", '""', '...', '``']
stopwords_list += ['--', 'xxxx']

In [104]:
def process_narrative(narrative):
    tokens = nltk.word_tokenize(narrative)
    stopwords_removed = [token.lower() for token in tokens if token.lower() not in stopwords_list]
    
    # adding line to remove all tokens with numbers and punctuation
    stopwords_punc_and_numbers_removed = [word for word in stopwords_removed if word.isalpha()]
    
    return stopwords_punc_and_numbers_removed  

In [105]:
processed_narr_1 = process_narrative(data.loc[0]['narrative'])
processed_narr_1[:5]

['purchase', 'order', 'days', 'shipping', 'amount']

You need to add lemmatization and then combine them into a single space separated string.

In [106]:
lemm = WordNetLemmatizer()

In [107]:
def concat_words(list_of_words):
    # remove any NaN's
    # list_of_words = [i for i in list if i is not np.nan]

    concat_words = ''
    for word in list_of_words:
        concat_words += word + ' '
    return concat_words.strip()

In [108]:
def make_lemma(list_of_words):
    # remove any NaN's
    list_of_words = [i for i in list_of_words if i is not np.nan]
    
    # lemmatize each word
    lemmatized_list = []
    for idx, word in enumerate(list_of_words):
        lemmatized_list.append(lemm.lemmatize(word))
    
    # make the list into a single string with the words separated by ' '
    concatenated_string = concat_words(lemmatized_list)
    return concatenated_string

In [109]:
make_lemma(processed_narr_1)[0:60]

'purchase order day shipping amount receive product week sent'

In [110]:
data

,class,narrative
0,credit,"-- -- -- -- -- 1. ) XXXX XXXX, XXXX a purchase..."
1,loan,-- -- -- -- -- Forwarded message -- -- -- -- -...
2,retail,"-- -- - XXXX, XXXX, XX/XX/2020 Payment Histori..."
3,credit,Make a point to try to make it to the homes


In [111]:
for i in range(len(data)):
    processed_narr = process_narrative(data['narrative'].loc[i])
    narr = make_lemma(processed_narr)
    data['narrative'].loc[i] = narr
data

,class,narrative
0,credit,purchase order day shipping amount receive pro...
1,loan,forwarded message date tue subject please inve...
2,retail,payment history missing credit report speciali...
3,credit,make point try make home


Now see if it works with a TF-IDF

In [112]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['narrative'])
feature_names = vectorizer.get_feature_names()
X_dense = X.T.todense()

In [113]:
X = X.todense()

In [114]:
X.shape

(4, 279)

In [115]:
pd.DataFrame(X, columns=feature_names)

,able,accordance,account,address,adjustment,adjustmentmerchandiserobert,affected,agreed,along,also,...,volume,waiting,way,wed,week,well,willing,without,would,york
0,0.000000,0.041232,0.000000,0.041232,0.041232,0.041232,0.000000,0.041232,0.000000,0.000000,...,0.041232,0.000000,0.000000,0.041232,0.065015,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.142107,0.000000,0.000000,0.000000,0.060081,0.000000,0.060081,0.060081,...,0.000000,0.000000,0.060081,0.000000,0.000000,0.120163,0.060081,0.000000,0.060081,0.060081
2,0.116211,0.000000,0.137433,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.058105,0.000000,0.000000,0.045811,0.000000,0.000000,0.116211,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [116]:
test_processed_df = pd.read_csv('../project_data/complaints_processed.csv')

In [117]:
test_processed_df.head(10)

,Unnamed: 0,product,narrative
0,0,credit_card,purchase order day shipping amount receive pro...
1,1,credit_card,forwarded message date tue subject please inve...
2,2,retail_banking,forwarded message cc sent friday pdt subject f...
3,3,credit_reporting,payment history missing credit report speciali...
4,4,credit_reporting,payment history missing credit report made mis...
5,5,credit_reporting,payment history missing credit report made mis...
6,6,credit_reporting,va date complaint experian credit bureau invol...
7,7,credit_reporting,account reported abbreviated name full name se...
8,8,credit_reporting,account reported abbreviated name full name se...
9,9,credit_reporting,usdoexxxx account reported abbreviated name fu...


# Practice with LDA

I think LDA needs Bag of Words format

In [118]:
# data = pd.DataFrame({
#     'class': ['credit', 'loan', 'retail', 'credit'],
#     'narrative': ['-- -- -- -- -- 1. ) XXXX XXXX, XXXX a purchase with EIO.com ( order # XXXX ) with 2 days shipping for the amount of {$57.00}. \n\n-- -- -- -- -- -2. ) Did not receive the product after a week, so I sent a followup email ; with the exact verbiage : " I paid for two days shipping and have not received my order \'\' on XXXX XXXX. \n\n-- -- -- -- -- 3. ) Company Responded on on XXXX XXXX : " Im sorry to inform you that due to unusually high order volume, your order has not shipped and XXXX not be for several weeks. XXXX has been out of stock since early XXXX due to high demand, although they are continuing to take orders and guaranteeing that we will receive orders we place with them in due time. As such we have masks on order but we do not have an exact shipping date from XXXX right now, however we can guarantee we will ship as soon as soon as XXXX delivers product to us. We have been getting small shipments and shipping those on a first come, first served basis, so we appreciate your patience as we fulfill these orders as quickly as we can. \n\nWe recommend keeping your order as you will lose your place in line if you cancel, and no other distributor has stock at the moment. If you do prefer to cancel, please note we ask that you do so via email, we will then cancel in accordance with our cancellation policy which was agreed to at checkout. \nXXXX, XXXX XXXX. \nElectronic Inventory Online -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- XXXX XXXX XXXX XXXX -- -- -- XXXX. ) XX/XX/XXXX, requested for the order to be canceled and a refund to be issued. \n\n-- -- -- 5. ) XX/XX/XXXX : XXXX canceled the order and sent a verification : XXXX XXXX XXXX Your order has been canceled Refunded Items XXXX XXXX Particulate Respirator, XXXX, XXXX eaches/pack 2 Refunded {$45.00} Subtotal {$45.00} Shipping {$12.00} Taxes {$0.00} Total {$57.00} USD Visa ( ending in XXXX XXXX {$57.00} Refund Shopify_payments - {$52.00} -- -- XX/XX/XXXX, Called in and disputed the amount. XXXX stated nothing needed to be submitted and they will address the issue. \n\n-- -- XX/XX/XXXX, XXXX recharged me for the same item after removing it on XX/XX/XXXX. Called back in to dispute this amount again. \n\nTRANSACTION REBILLMerchandiseRobert J. XXXX {$57.00} HTTPSEIOCOM.M, CA XXXX Purchased : Thu, XX/XX/2020 Posted : Wed, XXXX XXXX, 2020 Purchased By : XXXX XXXX Appears on the statement as : TRANSACTION REBILL HTTPSEIOCOM.M CA XXXX US XXXX - XX/XX/XXXX, followed up to see the status of the case. Submitted documentation showing XXXX canceled the order and was supposed to submit a refund. \n\n-- - XX/XX/XXXX, Called back in to speak to the manager on the case. He stated the dispute was ruled in favor of me, and the charge will be removed from my card. \n\n-- -- XX/XX/XXXX, Capital One removed the purchase off the bill. \nPURCHASE ADJUSTMENTMerchandiseRobert J. ... - {$57.00} Posted : Fri, XX/XX/2020 Purchased By : XXXX XXXX Appears on statement as : PURCHASE ADJUSTMENT XXXX XXXX -- XXXX XXXX Capital One recharges me the same amount. \nXX/XX/XXXX TRANSACTION REBILLMerchandiseRobert J. ... {$57.00} Purchased : Thu, XX/XX/2020 Posted : Mon, XX/XX/2020 Purchased By : XXXX XXXX Appears on statement as : TRANSACTION REBILL XXXX XXXX -- -- XX/XX/XXXX -- Called Capital One requested why this was recharge. They stated Visa ruled against my case. They pretended that they didn\'t remove the purchase or knew anything about the case manager ruling in my favor.',
#                   '-- -- -- -- -- Forwarded message -- -- -- -- - From : XXXX XXXX XXXX Date : Tue, XX/XX/XXXXat XXXX XXXX Subject : Please Investigate Comenity Bank Retailers card scam To : XXXX Sent from my XXXX Hello my name is XXXX XXXX, I am being scammed by Comenity bank a credit card provider for companies The Childrens place, New York & Co. , Forever 21 and Victoria Secret. My original credit from XXXX was {$500.00} Comenity bank then lowers my limit to {$300.00} and began to charge overage fees along with late fees. I then began to pay close attention to my other cards to find that my limits were also changed on them as well incurring overages and late fees. \nI reached out to the company Comenity bank they stated that they would change my credit limit to its original limits but did not. I reached out to them again and told them I will not summit any payment until my accounts are corrected. Comenity bank credit cards has impacted my credit scores plummeted to a negative status. Im currently paying the price due to the corruption in which has affected me in detrimental way. \nI am now in debt over {$2000.00} due to the company charging overage fees as well as late fees even through COVID-19. The initial credit limits are fluctuating tremendously and the company charges major fees on each of those 4 accounts. They are not willing to correct my account and was nervous when I said I had an attorney, that is the reason Im reaching out to you. Im a XXXX  employee of 18yrs and this company is ruining my credit. PLZ HELP!!!!! \n\nMy name is XXXX XXXX, contact info is ( XXXX ) XXXX XXXX and/or XXXX. Thank you!!',
#                  '-- -- - XXXX, XXXX, XX/XX/2020 Payment Histories Missing On My Credit Report -- -- Specialized Loan Servicing ( SLS ) has made the mistake to put my account under forbearance in XXXX 2020, without my authorization or knowledge ( As a matter of fact, I have automatic payment setup. In each month \'s XXXX, the monthly mortgage was paid in full ). \n\nI have noticed this issue ( my account was marked as " forbearance \'\' ) in XX/XX/2020 on my credit report ( when I tried to get a new home loan from another new bank ), I have contacted them immediately and asked them to fix this error and provide me a letter ( details please see below asks ). The " forbearance \'\' issue itself seemed to be fixed by now. However, on my credit report, the payment histories for XXXX, XXXX, XX/XX/2020 were all missing. My new bank will not be able to approve my new loan because of these issues of missing payment histories. \n\nI have contacted Specialized Loan Servicing more than 20 times since XX/XX/XXXX by phone to ask SLS to do 2 things : ( 1 ) report my XXXX, XXXX, XX/XX/2020 payment histories to XXXX, XXXX, XXXX  to fix the missing data issue. \n\n( 2 ) Provide me a letter with shows my account has never been under forbearance and my payment histories for all past 12 months.\n\nHowever, after waiting for more than 3 weeks, and countless emails / phone calls / talks with multiple supervisors, I have not been able to get either one of the above 2 things.\n\nWithout this issue fixed, My new bank can not process my new loan application, therefore I will need SLS \'s help immediately to get this fixed.',
#                  'Make a point to try to make it to the homes']
#     })

In [119]:
# def make_lemma_no_concat(list_of_words):
#     # remove any NaN's
#     list_of_words = [i for i in list_of_words if i is not np.nan]
    
#     # lemmatize each word
#     lemmatized_list = []
#     for idx, word in enumerate(list_of_words):
#         lemmatized_list.append(lemm.lemmatize(word))
    
#     return lemmatized_list

In [120]:
# for i in range(len(data)):
#     processed_narr = process_narrative(data['narrative'].loc[i])
#     narr = make_lemma_no_concat(processed_narr)
#     data['narrative'].loc[i] = narr
# data

... or maybe it needs it as CV format

In [122]:
data

,class,narrative
0,credit,purchase order day shipping amount receive pro...
1,loan,forwarded message date tue subject please inve...
2,retail,payment history missing credit report speciali...
3,credit,make point try make home


In [140]:
cv = CountVectorizer(ngram_range=[1,2])
X_cv_bigram = cv.fit_transform(data['narrative'])
feature_names = cv.get_feature_names()
# X_cv_bigram = X_cv_bigram.todense()

In [125]:
# X_cv_bigram.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 1],
        [2, 1, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [141]:
lda = LatentDirichletAllocation(n_components=3,random_state=0)
lda.fit(X_cv_bigram)

LatentDirichletAllocation(n_components=3, random_state=0)

In [138]:
def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f'Topic {topic_idx +1}',
                     fontdict={'fontsize': 30})
        ax.invert_yaxis()
        ax.tick_params(axis='both', which='major', labelsize=20)
        for i in 'top right left'.split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [143]:
feature_names = cv.get_feature_names()

In [179]:
for idx, topic in enumerate(lda.components_):
    print ("Topic ", idx, " ".join(feature_names[i] for i in topic.argsort()[:-6 - 1:-1]))

Topic  0 home try make make home try point point try
Topic  1 order credit company fee purchased shipping
Topic  2 payment new issue loan history payment history


In [221]:
for idx, topic in enumerate(lda.components_):
    print ("Topic ", idx, " ".join(feature_names[i] for i in topic.argsort()[:-7:-1]))

Topic  0 home try make make home try point point try
Topic  1 order credit company fee purchased shipping
Topic  2 payment new issue loan history payment history


In [151]:
topic[0:10]

array([2.33331215, 1.33331091, 1.33331091, 0.33334318, 0.33334318,
       3.3341189 , 0.33335116, 1.33331091, 1.33331091, 0.33335116])

In [154]:
topic10 = topic[0:10]
topic10

array([2.33331215, 1.33331091, 1.33331091, 0.33334318, 0.33334318,
       3.3341189 , 0.33335116, 1.33331091, 1.33331091, 0.33335116])

In [153]:
topic10.argsort()

array([3, 4, 6, 9, 1, 2, 7, 8, 0, 5], dtype=int64)

In [155]:
np.argsort(topic10)

array([3, 4, 6, 9, 1, 2, 7, 8, 0, 5], dtype=int64)

In [175]:
topic10.argsort()[0:-5]

array([3, 4, 6, 9, 1], dtype=int64)

In [177]:
topic10.argsort()[0:-5 - 2:-1]

array([], dtype=int64)

In [181]:
lda.score(X_cv_bigram)

-6785.706405176896

In [203]:
arg_arr = np.array([100, 300, 200, 400, 150])
arg_arr 

array([100, 300, 200, 400, 150])

In [204]:
arg_arr.argsort()

array([0, 4, 2, 1, 3], dtype=int64)

In [216]:
arg_arr.argsort()[:-2 : -1]

array([3], dtype=int64)